In [ ]:
def read_input(filename):
    char_list = []
    with open(filename, "r") as file:
        for line in file:
            for char in line:
                char_list.append(char)
    return char_list

In [ ]:
def compare_characters(char1, char2):
    if (char1.upper() == char2.upper()) & (char1.isupper() != char2.isupper()):
        return True
    return False

dabAcCaCBAcCcaDA

In [ ]:
def compress_polymer_method_1(input_list):
    polymers_removed = 1
    while polymers_removed > 0:
        polymers_removed = 0
        for i in range(len(input_list) - 1):
            if i >= len(input_list) - 1:
                break
            if compare_characters(input_list[i], input_list[i + 1]):
                input_list.pop(i)
                input_list.pop(i)
                polymers_removed += 1
    return input_list

In [ ]:
%%time
eg_list = read_input("input.txt")
result_list = compress_polymer_method_1(eg_list)
result = len(result_list)

In [ ]:
result